In [1]:
import os
import numpy as np
import numpy_indexed as npi

In [34]:
path = "/home/liwei/Desktop/origami_data/state2.obj"
with open(path) as file:
    vertice = []
    face = []
    while 1:
        line = file.readline()
        if not line:
            break
        strs = line.split(" ")
        if strs[0] == "v":
            vertice.append([float(strs[1]), float(strs[2]), float(strs[3])])
        if strs[0] == "f":
            del strs[0]
            strs = np.array(strs)
            strs = strs.astype(int)
            strs = strs - 1
            face.append(strs)

vertice = np.array(vertice)
vertice = np.around(vertice, decimals=2)
face = np.array(face)

line = []
line_to_vertice = []
for i in range(len(face)):
    line_temp = []
#     line_to_vertice_temp = []
    for j in range(1, len(face[i])-1):
        line_temp_vector1 = (vertice[face[i][j]] - vertice[face[i][j-1]]).tolist() #clockwise
        line_temp.append(line_temp_vector1)
        line_temp_vector2 = (vertice[face[i][j+1]] - vertice[face[i][j]]).tolist() #clockwise
        line_to_vertice.append([vertice[face[i][j]].tolist(), line_temp_vector1, line_temp_vector2])
    line_temp.append(line_temp_vector2) 
    line_temp_vector3 = (vertice[face[i][0]] - vertice[face[i][j+1]]).tolist()
    line_temp.append(line_temp_vector3)
    line_to_vertice.append([vertice[face[i][j+1]].tolist(), line_temp_vector2, line_temp_vector3])
    line_temp_vector4 = (vertice[face[i][1]] - vertice[face[i][0]]).tolist()
    line_to_vertice.append([vertice[face[i][0]].tolist(), line_temp_vector3, line_temp_vector4])

    line.append(line_temp)
#     line_to_vertice.append(line_to_vertice_temp)

line = np.array(line)
line_to_vertice = np.array(line_to_vertice)

def findbyrow(mat,row):
    return np.where((mat == row).all(1))[0]

In [35]:
# # b = np.delete(line_to_vertice,[1,2],axis = 1)
# # b = b.reshape(len(b),3)

# # print b
# a = np.array([2,3,4])
# # print b[a]
# b = line_to_vertice[a]
# print b
# c = np.delete(b,0,axis = 1)
# c = c.reshape(len(c)*2,3)
# d = np.array(c)
# print c,d,type(d),d.shape

# mat = np.array([[200,0,0],[100,0,0],[0,100,0],[100,100,0]])
# row = np.array([200,0,0])
# a = findbyrow(mat,row)
# # a= int(a[0])
# a = int(a)
# print a

a = line_to_vertice
b = line
print "aaa",a,a.shape
print "bbb",b

aaa [[[ 1.e+02  0.e+00  0.e+00]
  [ 1.e+02  0.e+00  0.e+00]
  [ 1.e+02  1.e+02  0.e+00]]

 [[ 2.e+02  1.e+02  0.e+00]
  [ 1.e+02  1.e+02  0.e+00]
  [ 0.e+00  1.e+02  0.e+00]]

 [[ 2.e+02  2.e+02  0.e+00]
  [ 0.e+00  1.e+02  0.e+00]
  [-2.e+02  0.e+00  0.e+00]]

 [[ 0.e+00  2.e+02  0.e+00]
  [-2.e+02  0.e+00  0.e+00]
  [ 0.e+00 -2.e+02  0.e+00]]

 [[ 0.e+00  0.e+00  0.e+00]
  [ 0.e+00 -2.e+02  0.e+00]
  [ 1.e+02  0.e+00  0.e+00]]

 [[ 2.e+02  1.e+02  0.e+00]
  [ 1.e+02  1.e+02  0.e+00]
  [-1.e+02  0.e+00  1.e-01]]

 [[ 1.e+02  1.e+02  1.e-01]
  [-1.e+02  0.e+00  1.e-01]
  [ 0.e+00 -1.e+02 -1.e-01]]

 [[ 1.e+02  0.e+00  0.e+00]
  [ 0.e+00 -1.e+02 -1.e-01]
  [ 1.e+02  1.e+02  0.e+00]]] (8, 3, 3)
bbb [list([[100.0, 0.0, 0.0], [100.0, 100.0, 0.0], [0.0, 100.0, 0.0], [-200.0, 0.0, 0.0], [0.0, -200.0, 0.0]])
 list([[100.0, 100.0, 0.0], [-100.0, 0.0, 0.1], [0.0, -100.0, -0.1]])]


In [9]:
class State(object):
   
    def __init__(self,path):
        self.path = path
        self.vertice, self.line, self.face, self.line_to_vertice = self.data_process(path)
    
    def data_process(self, path):
        '''
        This function is used to process .obj file, will return arrays of vertice, line, and face.
        Vertice: shape(num_v,3), num_v is the number of all vertices. Type: array.
        Line: shape(num_f,num_v_in_f[i],3), num_f is the number of all faces, 
            num_v_in_f[i] is the number of vertices in the ith face. Type: array.
        Face: shape(num_f,num_v_in_f[i]), num_f is  the number of all faces,
            num_v_in_face[i] is the number of vertices in the ith face. Type: array.
        Line to vertice: shape(num_v,3,3), num_v is the number of vertices. Type: array.
            restore num_v vertices, and each vertice connects two lines.
            format:[[vertice1,connect line1,connect line2],[vertice2,connect line2,connect line3]...]
        '''
        with open(path) as file:
            vertice = []
            face = []
            while 1:
                line = file.readline()
                if not line:
                    break
                strs = line.split(" ")
                if strs[0] == "v":
                    vertice.append([float(strs[1]), float(strs[2]), float(strs[3])])
                if strs[0] == "f":
                    del strs[0]
                    strs = np.array(strs)
                    strs = strs.astype(int)
                    strs = strs - 1
                    face.append(strs)

        vertice = np.array(vertice)
        vertice = np.around(vertice, decimals=2)
        face = np.array(face)

        line = []
        line_to_vertice = []
        for i in range(len(face)):
            line_temp = []
            for j in range(1, len(face[i])-1):
                line_temp_vector1 = (vertice[face[i][j]] - vertice[face[i][j-1]]).tolist() #clockwise
                line_temp.append(line_temp_vector1)
                line_temp_vector2 = (vertice[face[i][j+1]] - vertice[face[i][j]]).tolist() #clockwise
                line_to_vertice.append([vertice[face[i][j]].tolist(), line_temp_vector1, line_temp_vector2])
            line_temp.append(line_temp_vector2) 
            line_temp_vector3 = (vertice[face[i][0]] - vertice[face[i][j+1]]).tolist()
            line_temp.append(line_temp_vector3)
            line_to_vertice.append([vertice[face[i][j+1]].tolist(), line_temp_vector2, line_temp_vector3])
            line_temp_vector4 = (vertice[face[i][1]] - vertice[face[i][0]]).tolist()
            line_to_vertice.append([vertice[face[i][0]].tolist(), line_temp_vector3, line_temp_vector4])

            line.append(line_temp)
        line = np.array(line)
        line_to_vertice = np.array(line_to_vertice)

        return vertice, line, face, line_to_vertice
    
    def update_path(self, new_path):
        self.path = new_path

In [87]:
class CompareState(object):
    def __init__(self,state1,state2):
        self.state1 = state1
        self.state2 = state2
        self.crease_line, self.crease_point, self.manipulatible_vertices = self.getCreaseLineAndManipulatibleVertice()
        self.grasp_technique = self.getGraspTechnique()
        self.grasp_point, self.grasp_line, self.grasp_method = self.OptimalGraspTechinique()
        
    def diff(self,arrayA,arrayB):       
        #difference
        diff_a = npi.difference(arrayA,arrayB)
        diff_b = npi.difference(arrayB,arrayA)
 
        return diff_a, diff_b

    def findbyrow(self,mat,row):
        return np.where((mat == row).all(1))[0]
    
    def getCreaseLineAndManipulatibleVertice(self):
        '''
        logic: 1)search starts from the bottom face
               2)compare the difference between vertice[face[i]] of state1 and vertice[face[i]] of state2
                 a)if no difference -> face[i] hasn't been folded during last folding action;
                 b)if 2 new points added in face[i] of state2 -> crease is the line connecting the 2 new points;
                 c)if no new points in face[i] of state2 -> compare the difference between line[i] of state1, 2
                     -> crease is the line that in line[i] of state2 but not in line[i] of state1;
                 d)if 1 new points added in face[i] of state2 -> normalize line[i] in state1 and state2
                     -> crease is the line that in line[i] of state2 but not in line[i] of state1;
               3)search the next face, go back to 2)
        '''
        num_f1 = len(self.state1.face)
        num_f2 = len(self.state2.face) #num_f1 <= num_f2
        crease_line = []
        v1 = self.state1.vertice
        v2 = self.state2.vertice 
        crease_point = []
        
        for i in range(num_f1):
            f1 = self.state1.face[i]
            f2 = self.state2.face[i]
            l1 = self.state1.line[i]
            l2 = self.state2.line[i]
            diff_1, diff_2 = self.diff(v1[f1], v2[f2])
    
            if len(diff_1) == 0 and len(diff_2) == 0:
                print "No new added crease line in face ", i
                continue
                
            elif len(diff_2) == 2:
                crease_line = diff_2[1] - diff_2[0]
                crease_point.append(diff_2[0])
                crease_point.append(diff_2[1])
                
            elif len(diff_2) == 1:
                line_temp1 = []
                line_temp2 = []
                for j in range(len(l1)):
                    temp = l1[j] / np.linalg.norm(l1[j])
                    line_temp1.append(temp)
                for j in range(len(l2[i])):
                    temp = l2[j] / np.linalg.norm(l2[j])
                    line_temp2.append(temp)
                line_temp1 = np.array(line_temp1)
                line_temp2 = np.array(line_temp2)
                _, crease_line = self.diff(line_temp1, line_temp2)
                crease_line = crease_line.flatten()
                index = self.findbyrow(line_temp2,crease_line)
                index  = int(index)
                crease_point.append(v2[f2[index]])
                crease_point.append(v2[f2[index+1]])

                
            elif len(diff_2) == 0:
                _, crease_line = self.diff(l1, l2)
                crease_line = crease_line.flatten()
                index = self.findbyrow(l2,crease_line)
                index = int(index)
                crease_point.append(v2[f2[index]])
                crease_point.append(v2[f2[index+1]])

            manipulatible_vertices = diff_1
            crease_point = np.array(crease_point)
            
            if crease_line != []:
                break
                
        return crease_line, crease_point, manipulatible_vertices
    
    def getGraspTechnique(self):
        v1 = self.state1.vertice
        f1 = self.state1.face
        mv = self.manipulatible_vertices
        layer = int(mv[0][2] / 0.1)
        num_v = len(f1[layer])
        lv1 = self.state1.line_to_vertice[:num_v]
        lv1 = lv1.reshape(len(lv1)*3,3)
        grasp_technique = []
        

        for i in range(len(mv)):
            connect_lines = []
            index = self.findbyrow(v1,mv[i]) #mv[i]'s index in vertice
            index = int(index)
            index1 = self.findbyrow(lv1,mv[i])
            index1 = int(index1[0])
            connect_lines.append(lv1[index1+1]) 
            connect_lines.append(lv1[index1+2])
            
            count = 0
            for j in range(len(f1)):
                temp = np.where(f1[j]==index)
                temp = temp[0]
                if len(temp)>0:
                    count += 1

            if count == 1:
                action = "flex flip"
                grasp_technique.append([mv[i],connect_lines,action])
                
            elif count > 1:
                action = "scooping"
                v_temp = str(mv[i])
                grasp_technique.append([mv[i],connect_lines,action])                
                
        return grasp_technique

    def OptimalGraspTechinique(self):
        '''
        This function is used to determine optimal grasp point and grasp method.
        If only one grasp_technique, take the only one.
        If there are several choices, choose the one that minimize the angle between the grasp line and crease line.
        '''
        grasp = self.grasp_technique
        # transform 45 degree in the plane
        trans45 = np.array([[0.7071,-0.7071,0],[0.7071,0.7071,0],[0,0,1]])
        if len(grasp) == 1:
            grasp_point = grasp[0][0]
            grasp_line = grasp[0][1][0]
            grasp_method = grasp[0][2]
            return grasp_point, grasp_line, grasp_method
        elif len(grasp) > 1:
            norm_crease = self.crease_line / np.linalg.norm(self.crease_line)
            cosangle = []
            for i in range(len(grasp)):
                line = grasp[i][1][0]
                # grasp axis of flex flip is not align with the connect line, thus need to trans 45
                if grasp[i][2] == "flex flip":
                    line = np.dot(trans45,line)
                cosangle1 = line.dot(norm_crease)/np.linalg.norm(line)
                cosangle.append(cosangle1)
            index = cosangle.index(max(cosangle))
            grasp_point = grasp[index][0]
            grasp_line = grasp[index][1][0]
            grasp_method = grasp[index][2]  
            return grasp_point, grasp_line, grasp_method
        
    
    def getGraspPointPosition(self):
        '''
        This fuction is used to compute the target position of grasp point.
        Logic: grasp point and target position are symetric about the crease line.
        '''
        crease_line = self.crease_line
        
        return 0

In [88]:
path1 = "/home/liwei/Desktop/origami_data/state2.obj"
path2 = "/home/liwei/Desktop/origami_data/state3.obj"
state1 = State(path1)
state2 = State(path2)
compare = CompareState(state1,state2)

/home/liwei/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [89]:
a,b,c = compare.getCreaseLineAndManipulatibleVertice()
# print a,b,c
x = compare.getGraspTechnique()
print x

[[array([0., 0., 0.]), [array([   0., -200.,    0.]), array([100.,   0.,   0.])], 'flex flip'], [array([100.,   0.,   0.]), [array([100.,   0.,   0.]), array([100., 100.,   0.])], 'scooping']]


/home/liwei/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [90]:
y = compare.OptimalGraspTechinique()
print y


cos [0.7071067811865475, 1.0]
index 1
(array([100.,   0.,   0.]), array([100.,   0.,   0.]), 'scooping')
